## 간단한 웹 크롤링?

### 아무런 지식이 없을 경우

In [51]:

from collections import defaultdict

def get_txt(txt, start, end) : 
    cnt = txt.find(start)
    txt = txt[cnt + len(start) :]
    cnt =txt.find(end)
   
    return txt[ : cnt]

def get_link(txt, start, end) :
    stack = []
    while txt.find(start) != -1 :
        cnt = txt.find(start)
        txt = txt[cnt + len(start) :]
        cnt =txt.find(end)
        
        stack.append(txt[ : cnt])
        txt = txt[len(end) : ]
    return stack


# def get_score(txt, keyword ,start = 'a href=\"', end = '\"') :
#     txt = txt.lower()
#     keyword = keyword.lower()
#     score = 0.0
    
# #     while txt.find(start) != -1 :
# #         cnt = txt.find(start)
# #         temp =  txt[: cnt]    
# #         txt = txt[cnt + len(start) :]
# #         cnt =txt.find(end)
# #         txt = temp+ txt[cnt + len(end) : ]

#     while txt.find(keyword) != -1 :
#         cnt = txt.find(keyword)
        
#         txt = txt[cnt :]
        
#         if txt[0:len(keyword)] == txt[len(keyword):2*len(keyword)] :
        
#             while txt[0:len(keyword)] == txt[len(keyword):2*len(keyword)] :
#                     txt = txt[len(keyword) :]

#         else :
#             score +=1
#         txt = txt[ len(keyword) :]
#     return score

def get_score(txt, keyword) :
     
    txt = txt.lower()
    keyword = keyword.lower()
    score = 0.0
    
    while txt.find(keyword) != -1 :
        cnt = txt.find(keyword)
        
        a = txt[cnt-1]
        b = txt[cnt + len(keyword)]
        
        if a.isalpha() == False and b.isalpha() == False :
            score +=1
        
    
        txt = txt[cnt + len(keyword) +1:]
    return score

def getScore(page,word):
    import re
    return re.sub('[^a-z]+', '.', page.lower()).split('.').count(word.lower())

    
def solution(word, pages):
    answer = []
    temp = []
    dic_link = defaultdict(list)
    dic_score = defaultdict(list)
    dic_link_score = defaultdict(list)
    
    for d in pages :
              
        name = get_txt(d, start = '<meta property=\"og:url\" content=\"', end = '\"')
        temp.append(name)
        txt = get_txt(d, start = '<body>', end = '</body>')
        dic_link[name] += get_link(txt,start = 'a href=\"', end = '\"')
        dic_score[name] = get_score(txt,word) 

    
    for keys in temp :
        dic_link_score[keys] = 0.0
        
    for keys in dic_score.keys() :
        score = dic_score[keys]
        for v in dic_link[keys] : 
            total = len(dic_link[keys])   
            if v in dic_link_score :
         
                dic_link_score[v] += score/total
            
    n_max = -1
    for idx, v in enumerate(temp) :
        if dic_link_score[v] + dic_score[v] > n_max :
            answer = idx
            n_max = dic_link_score[v] + dic_score[v]

  
    print(dic_link)
    print(dic_score)
    print( dic_link_score)
   
    
    return answer

### 문자열과 조금의 정규표현식 응용한 경우

In [286]:
# 정규식 테스트 사이트 : https://regexr.com

In [317]:
import re

def solution(word, pages):
    webpage = {} # [{idx, link_stack, score, total_score }
    
    
    for idx, page in enumerate(pages) :
        page_name = page.split('<meta property=\"og:url\" content=\"')[1].split('\"')[0]
        
        link = []
        for link_data in page.split('<a href=\"')[1:] : 
            link.append(link_data.split('\"')[0])
         
        score = re.findall('[a-z]+',page.lower()).count(word.lower())
        
        webpage[page_name] = [idx, link,score, score]
    
    for keys in webpage.keys() :
        idx, link_stack, score, _ = webpage[keys]
        for link in link_stack : 
            if  link in webpage :
                webpage[link][3] += score/len(link_stack)
    
    n_max = -1
    
    for keys  in webpage.keys() :
        idx, _, _, total_score = webpage[keys]
        if total_score > n_max : 
            n_max = total_score
            answer = idx
    
    print(webpage)
    
    return answer
    
    
    

In [318]:
# 답 0 
solution("blind",["<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://a.com\"/>\n</head>  \n<body>\nBlind Lorem Blind ipsum dolor Blind test sit amet, consectetur adipiscing elit. \n<a href=\"https://b.com\"> Link to b </a>\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://b.com\"/>\n</head>  \n<body>\nSuspendisse potenti. Vivamus venenatis tellus non turpis bibendum, \n<a href=\"https://a.com\"> Link to a </a>\nblind sed congue urna varius. Suspendisse feugiat nisl ligula, quis malesuada felis hendrerit ut.\n<a href=\"https://c.com\"> Link to c </a>\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://c.com\"/>\n</head>  \n<body>\nUt condimentum urna at felis sodales rutrum. Sed dapibus cursus diam, non interdum nulla tempor nec. Phasellus rutrum enim at orci consectetu blind\n<a href=\"https://a.com\"> Link to a </a>\n</body>\n</html>"])

{'https://a.com': [0, ['https://b.com'], 3, 4.5], 'https://b.com': [1, ['https://a.com', 'https://c.com'], 1, 4.0], 'https://c.com': [2, ['https://a.com'], 1, 1.5]}


0

In [319]:
# 답 1
solution("Muzi",["<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://careers.kakao.com/interview/list\"/>\n</head>  \n<body>\n<a href=\"https://programmers.co.kr/learn/courses/4673\"></a>#!MuziMuzi!)jayg07con&&\n\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://www.kakaocorp.com\"/>\n</head>  \n<body>\ncon%\tmuzI92apeach&2<a href=\"https://hashcode.co.kr/tos\"></a>\n\n\t^\n</body>\n</html>"])

{'https://careers.kakao.com/interview/list': [0, ['https://programmers.co.kr/learn/courses/4673'], 0, 0], 'https://www.kakaocorp.com': [1, ['https://hashcode.co.kr/tos'], 1, 1]}


1

In [47]:
def solution(word, pages):
    answer = 0    
    for idx, page in enumerate(pages) :
        print(page)
    return answer

In [215]:
data = '<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://careers.kakao.com/interview/list\"/>\n</head>  \n<body>\n<a href=\"https://programmers.co.kr/learn/courses/4673\"></a>#!MuziMuzi!)jayg07con&&\n\n</body>\n</html>", "<html lang=\"ko\" xml:lang=\"ko\" xmlns=\"http://www.w3.org/1999/xhtml\">\n<head>\n  <meta charset=\"utf-8\">\n  <meta property=\"og:url\" content=\"https://www.kakaocorp.com\"/>\n</head>  \n<body>\ncon%\tmuzI92apeach&2<a href=\"https://hashcode.co.kr/tos\"></a>\n\n\t^\n</body>\n</html>'

In [196]:
 print(p.match('azzzzz1'))

<re.Match object; span=(0, 6), match='azzzzz'>


In [292]:
p = re.compile('[A-Za-z]+')
a = p.sub( ' ', data.lower())
data


'<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">\n<head>\n  <meta charset="utf-8">\n  <meta property="og:url" content="https://careers.kakao.com/interview/list"/>\n</head>  \n<body>\n<a href="https://programmers.co.kr/learn/courses/4673"></a>#!MuziMuzi!)jayg07con&&\n\n</body>\n</html>", "<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">\n<head>\n  <meta charset="utf-8">\n  <meta property="og:url" content="https://www.kakaocorp.com"/>\n</head>  \n<body>\ncon%\tmuzI92apeach&2<a href="https://hashcode.co.kr/tos"></a>\n\n\t^\n</body>\n</html>'

In [241]:
print(a)

 html lang ko xml lang ko xmlns http www w org xhtml head meta charset utf meta property og url content https careers kakao com interview list head body a href https programmers co kr learn courses a muzimuzi jayg con body html html lang ko xml lang ko xmlns http www w org xhtml head meta charset utf meta property og url content https www kakaocorp com head body con muzi apeach a href https hashcode co kr tos a body html 


In [242]:
print(a.split(' '))

['', 'html', 'lang', 'ko', 'xml', 'lang', 'ko', 'xmlns', 'http', 'www', 'w', 'org', 'xhtml', 'head', 'meta', 'charset', 'utf', 'meta', 'property', 'og', 'url', 'content', 'https', 'careers', 'kakao', 'com', 'interview', 'list', 'head', 'body', 'a', 'href', 'https', 'programmers', 'co', 'kr', 'learn', 'courses', 'a', 'muzimuzi', 'jayg', 'con', 'body', 'html', 'html', 'lang', 'ko', 'xml', 'lang', 'ko', 'xmlns', 'http', 'www', 'w', 'org', 'xhtml', 'head', 'meta', 'charset', 'utf', 'meta', 'property', 'og', 'url', 'content', 'https', 'www', 'kakaocorp', 'com', 'head', 'body', 'con', 'muzi', 'apeach', 'a', 'href', 'https', 'hashcode', 'co', 'kr', 'tos', 'a', 'body', 'html', '']


In [293]:
b = p.findall(data)
print(b)

['html', 'lang', 'ko', 'xml', 'lang', 'ko', 'xmlns', 'http', 'www', 'w', 'org', 'xhtml', 'head', 'meta', 'charset', 'utf', 'meta', 'property', 'og', 'url', 'content', 'https', 'careers', 'kakao', 'com', 'interview', 'list', 'head', 'body', 'a', 'href', 'https', 'programmers', 'co', 'kr', 'learn', 'courses', 'a', 'MuziMuzi', 'jayg', 'con', 'body', 'html', 'html', 'lang', 'ko', 'xml', 'lang', 'ko', 'xmlns', 'http', 'www', 'w', 'org', 'xhtml', 'head', 'meta', 'charset', 'utf', 'meta', 'property', 'og', 'url', 'content', 'https', 'www', 'kakaocorp', 'com', 'head', 'body', 'con', 'muzI', 'apeach', 'a', 'href', 'https', 'hashcode', 'co', 'kr', 'tos', 'a', 'body', 'html']


In [262]:
dir(1)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']